In [15]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd


In [16]:
load_dotenv()

url = "https://financialmodelingprep.com/stable/search-name"
params = {
    "apikey": os.getenv("FMP_KEY"),
    "exchange": "NASDAQ"
}

In [17]:
df = pd.read_csv("./data/FMP ticker mapping - not found v3.csv", header=None)
df.columns = ["ticker", "name", 'clean name']
print(df.head(), len(df))

       ticker                             name        clean name
0        QFIN               360 DigiTech, Inc.          Digitech
1  CGPOWER_IN  CG Power & Industrial Solutions          CG Power
2         PAC                 Pacific Airports  Pacific Airports
3         NOK                            Nokia             Nokia
4        UVSP                          Univest           Univest 117


In [19]:
def clean(s):
    def fix_mojibake_and_convert_to_ascii(s):
        s = str(s)
        try:
            # Step 1: Re-encode as bytes using the encoding that misinterpreted UTF-8
            bytes_data = s.encode('latin1')
            # Step 2: Decode properly using UTF-8
            fixed = bytes_data.decode('utf-8')
        except UnicodeEncodeError:
            # If encoding fails, fallback to original
            fixed = s
        except UnicodeDecodeError:
            # If decoding fails, fallback to original
            fixed = s

        # Step 3: Convert to ASCII (remove accents etc.)
        import unicodedata
        ascii_str = unicodedata.normalize('NFKD', fixed).encode('ascii', 'ignore').decode('ascii')
        return ascii_str
    s = fix_mojibake_and_convert_to_ascii(s).lower().replace('.','').replace(',','').replace("'",'')
    s = s.replace(' ltd','').replace(' inc','').replace('nv','').replace(' co', '').replace('-',' ').replace(' sa', '')
    return s.replace(' group', '').replace(' corporation', '').replace(' corp', '')
    


df['clean name'] = df['clean name'].apply(clean)
print(df.head())

       ticker                             name        clean name
0        QFIN               360 DigiTech, Inc.          digitech
1  CGPOWER_IN  CG Power & Industrial Solutions          cg power
2         PAC                 Pacific Airports  pacific airports
3         NOK                            Nokia             nokia
4        UVSP                          Univest           univest


In [20]:
symbols, names, tickers, fmp_names, exchanges = [], [], [], [], []

counter = 0

for company, name, ticker in zip(df['clean name'], df['name'], df['ticker']):
    print(company)
    
    params['query'], params['exchange'] = company, "NASDAQ"
    # r = requests.get(url, params=params)
    try:
    #     if len(r.json()):
    #         for res in r.json():
    #             names.append(name)
    #             tickers.append(ticker)
    #             symbols.append(res['symbol'])
    #             fmp_names.append(res['name'])
    #             exchanges.append(res['exchange'])
    #     else:
    #         params['exchange'] = "NYSE"
    #         r = requests.get(url, params=params)
    #         if len(r.json()):
    #             for res in r.json():
    #                 names.append(name)
    #                 tickers.append(ticker)
    #                 symbols.append(res['symbol'])
    #                 fmp_names.append(res['name'])
    #                 exchanges.append(res['exchange'])
    #         else:
        params['exchange'] = ""
        r = requests.get(url, params=params)
        if len(r.json()) == 0:
            raise Exception("No data")
        for res in r.json():
            names.append(name)
            tickers.append(ticker)
            symbols.append(res['symbol'])
            fmp_names.append(res['name'])
            exchanges.append(res['exchange'])
            
    except Exception as e:
        counter += 1
        print(e, counter)
        names.append(company)
        tickers.append(ticker)
        symbols.append('')
        fmp_names.append('')
        exchanges.append('')
#         
    
print(symbols)
        

digitech
cg power
pacific airports
No data 1
nokia
univest
toppan
unibail rodamco
No data 2
orthofix
upm
spencer
gladstone
immunic
liberty
bark
trustco bank
northwestern
raytheon technologies
No data 3
resources
tanger
verallia
shanghai
hygon information
No data 4
goodwe
chr
acs
galp energia
powszechny
super league
empyrean
isoftstone
No data 5
aena
mindray
range
tf1
No data 6
byhealth
No data 7
sinclair
gemdale
kyverna
biohaven
bmw
warehouses
fastighets
ams
sbb
No data 8
tingyi
l&t
zhongsheng
nan
barnes
willscot
xperi
innovex
txo
No data 9
ew
tpi
sohu
coast bancshares
si bone
No data 10
dsm
iestor
No data 11
ly
burke
resonac
No data 12
optimizerx
eiri
modivcare 
affle
acres
oncocyte
honest
stellus
guaranty
firstvings
No data 13
compass
stroeer
No data 14
citizens
dare
america airports
No data 15
broadwind
j
fidus
eista
No data 16
advanced
mayville
metals
y mabs
No data 17
avient
idex
cedar
duke
myt
lion
omv
munich
No data 18
rbc
angi
culp
weave
#value!
No data 19
kore
first bank
lulus

In [21]:
df = pd.DataFrame({
    'Ticker' : tickers,
    'Name' : names,
    'FMP name': fmp_names,
    'FMP symbol' : symbols,
    'Exchange' : exchanges
})
df.to_csv(r'data/FMP extracted6.csv', index=False)

In [27]:
print(counter)
exit()

120
